# GWAS integration: TWAS and MR

## Introduction

This module provides software implementations for transcriptome-wide association analysis (TWAS), and Mendelian Randomization using fine-mapping instrumental variables (IV). The procedures implements the MR procedure described in Zhang et al 2020 for "causal" efffects estimation and model validation, with the unit of analysis being a single gene-trait pair.

This procedure is based on the SuSiE-TWAS workflow --- it assumes that xQTL fine-mapping has been performed (to be used for both TWAS and MR) and moleuclar traits prediction weights pre-computed (to be used for TWAS). Cross validation for TWAS weights is optional but highly recommended. 

GWAS data required are GWAS summary statistics and LD matrix for the region of interest.

### Step 1: TWAS 

1. Extract GWAS z-score for region of interest and corresponding LD matrix.
2. (Optional) perform allele matching QC for the LD matrix with summary stats.
3. Process weights: for LASSO, Elastic Net and mr.ash we have to take the weights as is for QTL variants overlapping with GWAS variants. For SuSiE weights it can be adjusted to exactly match GWAS variants.
4. Perofrm TWAS test for multiple sets of weights. 
5. For each gene, filter TWAS results by keeping the best model selected by CV. Drop the genes that don't show good evidence of TWAS prediction weights.

### Step 2: MR for candidate genes

1. Limit MR only to those showing some evidence of TWAS significance AND have strong instrumental variable (fine-mapping PIP or CS). 
2. Use fine-mapped xQTL with GWAS data to perform MR. 
3. For multiple IV, aggregate individual IV estimates using a fixed-effect meta-analysis procedure.
4. Identify and exclude results with severe violations of the exclusion restriction (ER) assumption.

### Input

- GWAS summary statistics files (one or multiple GWAS)
- LD reference meta-data (see our LD reference preparation document for more details FIXME Tosin has to do this)
- xQTL fine-mapping model objects 
- `susie_path`: List of file paths for susie results.
- `GWAS_path`: GWAS summary statistics path (for loading the GWAS sumstats dataframe).
- `LD_path`: LD block matrix path (list of file paths of LD block).
- `TAD_path`: TAD region path (dataframe of TAD region for each gene, used to subset the LD matrix).

### Output

TWAS 

FIXME this is incorrect for now.

- Each row corresponds to a single SNP inferred as a member of a signal cluster, with columns including:
   - `snp`: SNP name.
   - `beta_eQTL`: eQTL effect.
   - `se_eQTL`: Standard error of estimated eQTL effect.
   - `beta_GWAS`: GWAS effect.
   - `se_GWAS`: Standard error of GWAS effect.
   - `cluster`: Signal cluster ID (credible sets index).
   - `pip`: SNP posterior inclusion probability (PIP).
   - `gene_id`: Gene name.


#### Output File Format

MR

-  The output includes the following columns for each gene:
    - `gene_id`: Gene name.
    - `num_cluster`: Number of credible sets of the gene.
    - `num_instruments`: Number of instruments included in the gene.
    - `spip`: Sum of PIP for credible sets of each gene.
    - `grp_beta`: Signal-level estimates, combining SNP-level estimates from member SNPs weighted by their PIPs.
    - `grp_se`: Standard error of signal-level estimates.
    - `meta`: Gene-level estimate of the causal effect, aggregating signal-level estimates using a fixed-effect meta-analysis model.
    - `se_meta`: Standard error of the gene-level estimate of the causal effect.
    - `Q`: Cochran’s Q statistic.
    - `I2`: $I^2$ statistics

In [ ]:
[global]
# Workdir
parameter: susie_path = paths
parameter: cwd = path("output")
parameter: container = ''
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
parameter: job_size = 100
parameter: walltime = "1h"
parameter: mem = "16G"
parameter: numThreads = 20

In [ ]:
[twas_z_format_1]

import pandas as pd
s_path = pd.read_csv(susie_path, header=None)
# split_path = s_path[0].str.split(".", expand=True)
# ID = pd.DataFrame({'ID': split_path[1]})
# path = pd.DataFrame({'path': s_path[0]})
# input_df = pd.concat([ID, path], axis=1)
input_df = s_path.values.tolist()

parameter: GWAS_path = path
parameter: LD_path = path
parameter: TAD_path = path
#input: susie_path
input: [x for x in input_df], group_by = 1
output: f'{cwd}/{step_name[:-2]}/{_input:bnn}.twas_z.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand= "${ }", stderr = f'{_output:nn}.stderr', stdout = f'{_output:nn}.stdout'

    options(bedtools.path = "/home/aw3600/bedtools2/bin/")
    library(pecotmr)
    library(dplyr)
    library(stringr)
    library(data.table)

    susie_path = str_split(${_input:ar},"\\.",simplify=T)[,2]%>%cbind(.,${_input:ar})%>%data.frame()%>%setNames(c("ID","path"))
    TAD_region = fread(${TAD_path:ar})%>%rename("ID"="gene_id")
    gene_name = susie_path$ID
    region = TAD_region%>%filter(ID==gene_name)%>%rename("chr"="#chr")
    chr = str_sub(region$chr,4)
    AD_dataset = fread(paste0(${GWAS_path:ar},"/ADGWAS_Bellenguez_2022.",chr,"/ADGWAS2022.chr",chr,".sumstat.tsv",sep=""))
    GWAS_data = AD_dataset%>%mutate(chr = paste0("chr",chromosome))%>%mutate(pos = as.character(position))%>%select(-chromosome)%>%rename("A1"="ref","A2"="alt")%>%mutate(z=beta/se)%>%select(-position)
    LD_list = read.table(${LD_path:ar},header=F,sep="\t")
    LD_block_path = str_split(LD_list$V1,"_",simplify=T)%>%cbind(.,LD_list)%>%rename("chr"="1","start"="2","end"="3","variants"="V1","path" = "V2")
    twas_z = twas_scan(susie_path, region, GWAS_data, LD_block_path)
    saveRDS(twas_z, ${_output:ar}, compress='xz')

In [ ]:
[twas_z_format_2]
# Path to the input molecular phenotype data.
input: group_by = "all"
output: f'{cwd}/{step_name[:-2]}/twas_z_files.txt'
python: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container, entrypoint = entrypoint
import pandas as pd
pd.DataFrame({"output" : [$[_input:ar,]]}).to_csv("$[_output]",index = False ,header = False, sep = "\t")

In [ ]:
[twas_candidate_gene]
parameter: weights_path = path
parameter: pval_threshold = 0.05
parameter: cpip_cutoff = 0.5
input: weights_path
output: cand_genes = f'{cwd:a}/{step_name}.rds',
        mr_output = f'{cwd:a}/mr_output.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    library(data.table)
    library(dplyr)
    library(stringr)
    library(qvalue)
    library(doMC)
    library(pecotmr)

    ####
    susie_list = fread(${susie_path:ar},header=F)
    susie_path = str_split(susie_list$V1,"\\.",simplify=T)[,2]%>%cbind(.,susie_list)%>%setNames(c("ID","path"))

    #detectCores()
    registerDoMC(100)
    weights_file = fread(${_input:ar},header=F)
    weights_file_path = str_split(weights_file$V1,"\\.",simplify=T)[,2]%>%cbind(.,weights_file)%>%setNames(c("ID","path"))
    #ptm = proc.time()
    gene_pq = foreach(k=1:dim(weights_file)[1], .combine=rbind) %dopar% {
       weights_pq = readRDS(weights_file_path$path[k])$gene_weights_pq
    }
    #proc.time() - ptm
    padj = apply(gene_pq[,c("susie_pval","lasso_pval","enet_pval","mr_ash_pval")],2,function(x) p.adjust(x, method = "bonferroni",))%>%data.frame()
    gene_pq_adj = gene_pq%>%cbind(.,padj)
    names(gene_pq_adj)[15:18] = c("susie_pval_adj","lasso_pval_adj","enet_pval_adj","mr_ash_pval_adj")
    gene.ref = read.table("/mnt/vast/hpc/csg/molecular_phenotype_calling/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list",header=F)
    gene_pq_adj$gene_id = gene.ref[match(gene_pq_adj$gene_name,gene.ref$V4),"V5"]

    cand_genes = NULL
    cand_genes$gene_pq_adj = gene_pq_adj
    cand_genes$susie= cand_genes$gene_pq_adj%>%filter(susie_pval<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$lasso= cand_genes$gene_pq_adj%>%filter(lasso_pval<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$enet= cand_genes$gene_pq_adj%>%filter(enet_pval<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$mr_ash= cand_genes$gene_pq_adj%>%filter(mr_ash_pval<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$susie_adj= cand_genes$gene_pq_adj%>%filter(susie_pval_adj<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$lasso_adj= cand_genes$gene_pq_adj%>%filter(lasso_pval_adj<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$enet_adj= cand_genes$gene_pq_adj%>%filter(enet_pval_adj<${pval_threshold})%>%select(gene_name,gene_id)
    cand_genes$mr_ash_adj= cand_genes$gene_pq_adj%>%filter(mr_ash_pval_adj<${pval_threshold})%>%select(gene_name,gene_id)

    # cand_genes$susie_format_input = twas_mr_format_input(cand_genes$susie$gene_name,${susie_path:ar},weights_file_path)
    # cand_genes$lasso_format_input = twas_mr_format_input(cand_genes$lasso$gene_name,${susie_path:ar},weights_file_path)
    # cand_genes$enet_format_input = twas_mr_format_input(cand_genes$enet$gene_name,${susie_path:ar},weights_file_path)
    # cand_genes$mr_ash_format_input = twas_mr_format_input(cand_genes$mr_ash$gene_name,${susie_path:ar},weights_file_path)
    cand_genes$susie_adj_format_input = twas_mr_format_input(cand_genes$susie_adj,susie_path,weights_file_path)
    cand_genes$lasso_adj_format_input = twas_mr_format_input(cand_genes$lasso_adj,susie_path,weights_file_path)
    cand_genes$enet_adj_format_input = twas_mr_format_input(cand_genes$enet_adj,susie_path,weights_file_path)
    cand_genes$mr_ash_adj_format_input = twas_mr_format_input(cand_genes$mr_ash_adj,susie_path,weights_file_path)

    saveRDS(cand_genes, ${_output[0]:ar}, compress='xz')

    mr_output = NULL
    # mr_output$susie = fine_mr(cand_genes$susie_format_input,${cpip_cutoff})
    # mr_output$lasso = fine_mr(cand_genes$lasso_format_input,${cpip_cutoff})
    # mr_output$enet = fine_mr(cand_genes$enet_format_input,${cpip_cutoff})
    # mr_output$mr_ash = fine_mr(cand_genes$mr_ash_format_input,${cpip_cutoff})
    if(!is.null(cand_genes$susie_adj_format_input))
    {mr_output$susie_adj = fine_mr(cand_genes$susie_adj_format_input,${cpip_cutoff})}
    if(!is.null(cand_genes$lasso_adj_format_input))
    {mr_output$lasso_adj = fine_mr(cand_genes$lasso_adj_format_input,${cpip_cutoff})}
    if(!is.null(cand_genes$enet_adj_format_input))
    {mr_output$enet_adj = fine_mr(cand_genes$enet_adj_format_input,${cpip_cutoff})}
    if(!is.null(cand_genes$mr_ash_adj_format_input))
    {mr_output$mr_ash_adj = fine_mr(cand_genes$mr_ash_adj_format_input,${cpip_cutoff})}

    saveRDS(mr_output, ${_output[1]:ar}, compress='xz')